# [Effective TensorFlow for Non-Experts (Google IO '17)](http://youtu.be/?v=5DknTFbcGVM)
c:\Users\hcche\Documents\ML YouTube and Moocs\Machine Learning\Misc of ML and AI\Inception slim models API\Effective TensorFlow for Non-Experts (Google I_O '17)-5DknTFbcGVM.mp4                      
# [Estimator demo using Automobile dataset](https://gist.github.com/martinwicke/6838c23abdc53e6bcda36ed9f40cff39)
[Jupyter Notebook martinwicke/automobile.ipynb](https://goo.gl/0OgXiL)




In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

In [2]:
# First thing to do: Download https://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data

In [3]:
# We're using pandas to read the CSV file. This is easy for small datasets, but for large and complex datasets,
# tensorflow parsing and processing functions are more powerful.
import pandas as pd
import numpy as np

In [4]:
# The CSV file does not have a header, so we have to fill in column names.
names = [
    'symboling', 
    'normalized-losses', 
    'make', 
    'fuel-type', 
    'aspiration',
    'num-of-doors',
    'body-style',
    'drive-wheels',
    'engine-location',
    'wheel-base',
    'length',
    'width',
    'height',
    'curb-weight',
    'engine-type',
    'num-of-cylinders',
    'engine-size',
    'fuel-system',
    'bore',
    'stroke',
    'compression-ratio',
    'horsepower',
    'peak-rpm',
    'city-mpg',
    'highway-mpg',
    'price',
]

# We also have to specify dtypes.
dtypes = {
    'symboling': np.int32, 
    'normalized-losses': np.float32, 
    'make': str, 
    'fuel-type': str, 
    'aspiration': str,
    'num-of-doors': str,
    'body-style': str,
    'drive-wheels': str,
    'engine-location': str,
    'wheel-base': np.float32,
    'length': np.float32,
    'width': np.float32,
    'height': np.float32,
    'curb-weight': np.float32,
    'engine-type': str,
    'num-of-cylinders': str,
    'engine-size': np.float32,
    'fuel-system': str,
    'bore': np.float32,
    'stroke': np.float32,
    'compression-ratio': np.float32,
    'horsepower': np.float32,
    'peak-rpm': np.float32,
    'city-mpg': np.float32,
    'highway-mpg': np.float32,
    'price': np.float32,    
}

In [5]:
# Read the file.
df = pd.read_csv('imports-85.data', names=names, dtype=dtypes, na_values='?')

In [6]:
# Some rows don't have price data, we can't use those.
df = df.dropna(axis='rows', how='any', subset=['price'])

In [7]:
# Fill missing values in continuous columns with zeros instead of NaN.
float_columns = [k for k,v in dtypes.items() if v == np.float32]
df[float_columns] = df[float_columns].fillna(value=0., axis='columns')
# Fill missing values in continuous columns with '' instead of NaN (NaN mixed with strings is very bad for us).
string_columns = [k for k,v in dtypes.items() if v == str]
df[string_columns] = df[string_columns].fillna(value='', axis='columns')

In [8]:
# Split the data into a training set and an eval set.
training_data = df[:160]
eval_data = df[160:]

# Separate input features from labels
training_label = training_data.pop('price')
eval_label = eval_data.pop('price')

In [9]:
# Now we can start using some TensorFlow.
import tensorflow as tf
print('please make sure that version >= 1.2:')
print(tf.__version__)

please make sure that version >= 1.2:
1.4.0


In [10]:
# Make input function for training: 
#   num_epochs=None -> will cycle through input data forever
#   shuffle=True -> randomize order of input data
training_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=training_data, y=training_label, 
    batch_size=64, shuffle=True, num_epochs=None)

# Make input function for evaluation:
#   shuffle=False -> do not randomize input data
eval_input_fn = tf.estimator.inputs.pandas_input_fn(
    x=eval_data, y=eval_label, batch_size=64, shuffle=False)

In [13]:
# Describe how the model should interpret the inputs. 
# The names of the feature columns have to match the names
# of the series in the dataframe.
# [ ] 我們玩 WH300 就遇到了類似的問題， catagorical column 要
先把 string 置換成數字才能餵給選定的 classifier, 但又怕數字的算數意義會不
會被 classifier 誤會成某種特徵？以 make 為例，用 hash_bucket 就對了。
我想看看泡製過的 make 是甚麼樣子？


symboling = tf.feature_column.numeric_column('symboling')
normalized_losses = tf.feature_column.numeric_column('normalized-losses')
make = tf.feature_column.categorical_column_with_hash_bucket('make', 50)
fuel_type = tf.feature_column.categorical_column_with_vocabulary_list('fuel-type', vocabulary_list=['diesel', 'gas'])
aspiration = tf.feature_column.categorical_column_with_vocabulary_list('aspiration', vocabulary_list=['std', 'turbo'])
num_of_doors = tf.feature_column.categorical_column_with_vocabulary_list('num-of-doors', vocabulary_list=['two', 'four'])
body_style = tf.feature_column.categorical_column_with_vocabulary_list('body-style', vocabulary_list=['hardtop', 'wagon', 'sedan', 'hatchback', 'convertible'])
drive_wheels = tf.feature_column.categorical_column_with_vocabulary_list('drive-wheels', vocabulary_list=['4wd', 'rwd', 'fwd'])
engine_location = tf.feature_column.categorical_column_with_vocabulary_list('engine-location', vocabulary_list=['front', 'rear'])
wheel_base = tf.feature_column.numeric_column('wheel-base')
length = tf.feature_column.numeric_column('length')
width = tf.feature_column.numeric_column('width')
height = tf.feature_column.numeric_column('height')
curb_weight = tf.feature_column.numeric_column('curb-weight')
engine_type = tf.feature_column.categorical_column_with_vocabulary_list('engine-type', ['dohc', 'dohcv', 'l', 'ohc', 'ohcf', 'ohcv', 'rotor'])
num_of_cylinders = tf.feature_column.categorical_column_with_vocabulary_list('num-of-cylinders', ['eight', 'five', 'four', 'six', 'three', 'twelve', 'two'])
engine_size = tf.feature_column.numeric_column('engine-size')
fuel_system = tf.feature_column.categorical_column_with_vocabulary_list('fuel-system', ['1bbl', '2bbl', '4bbl', 'idi', 'mfi', 'mpfi', 'spdi', 'spfi'])
bore = tf.feature_column.numeric_column('bore')
stroke = tf.feature_column.numeric_column('stroke')
compression_ratio = tf.feature_column.numeric_column('compression-ratio')
horsepower = tf.feature_column.numeric_column('horsepower')
peak_rpm = tf.feature_column.numeric_column('peak-rpm')
city_mpg = tf.feature_column.numeric_column('city-mpg')
highway_mpg = tf.feature_column.numeric_column('highway-mpg')

In [14]:
linear_features = [symboling, normalized_losses, make, fuel_type, aspiration, num_of_doors,
                   body_style, drive_wheels, engine_location, wheel_base, length, width,
                   height, curb_weight, engine_type, num_of_cylinders, engine_size, fuel_system,
                   bore, stroke, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg]

In [ ]:
regressor = tf.contrib.learn.LinearRegressor(feature_columns=linear_features)

\ 下面查出一堆 classifier, Estimator, Regressor 都不知怎麼區分的？ Stackoverflow 上有人問了也沒人回答 <br>
\ https://stackoverflow.com/questions/43401721/difference-between-dnnlinearcombinedestimator-regressor-classifier-tensorflo<br>
\ 還太新了？<br>
\__main\__ :> tf.contrib.learn dir . cr 
\['BaseEstimator', 'DNNClassifier', 'DNNEstimator', 'DNNLinearCombinedClassifier', 'DNNLinearCombinedEstimator', 'DNNLinearCombinedRegressor', __'DNNRegressor'__, 'DynamicRnnEstimator', 'Estimator', 'Evaluable', 'Experiment', 'ExportStrategy', 'Head', 'InputFnOps', 'KMeansClustering', 'LinearClassifier', 'LinearEstimator', __'LinearRegressor'__, 'LogisticRegressor', 'MetricSpec', 'ModeKeys', 'ModelFnOps', 'NanLossDuringTrainingError', 'NotFittedError', 'PredictionKey', 'ProblemType', 'RunConfig', 'SKCompat', 'SVM', 'TaskType', 'Trainable', ...snip ..., 'binary_svm_head', 'build_parsing_serving_input_fn', 'datasets', 'evaluate', 'extract_dask_data', 'extract_dask_labels', 'extract_pandas_data', 'extract_pandas_labels', 'extract_pandas_matrix', 'graph_actions', 'head', 'infer', 'infer_real_valued_columns_from_input', 'infer_real_valued_columns_from_input_fn', 'io', __'learn_runner'__, 'make_export_strategy', 'models', 'monitors', 'multi_class_head', 'multi_head', 'multi_label_head', 'no_op_train_fn', 'ops', 'poisson_regression_head', 'preprocessing', 'read_batch_examples', 'read_batch_features', 'read_batch_record_features', 'read_keyed_batch_examples', 'read_keyed_batch_examples_shared_queue', 'read_keyed_batch_features', 'read_keyed_batch_features_shared_queue', 'regression_head', 'run_feeds', 'run_n', 'train', 'utils']
OK exit
OK 


In [ ]:
regressor.fit(input_fn=training_input_fn, steps=10000)

In [ ]:
regressor.evaluate(input_fn=eval_input_fn)

In [15]:
dnn_features = [
    #numerical features
    symboling, normalized_losses, wheel_base, length, width, height, curb_weight, engine_size,
    bore, stroke, compression_ratio, horsepower, peak_rpm, city_mpg, highway_mpg,    
    # densify categorical features:
    tf.feature_column.indicator_column(make),
    tf.feature_column.indicator_column(fuel_type),
    tf.feature_column.indicator_column(aspiration),
    tf.feature_column.indicator_column(num_of_doors),
    tf.feature_column.indicator_column(body_style),
    tf.feature_column.indicator_column(drive_wheels), 
    tf.feature_column.indicator_column(engine_location),
    tf.feature_column.indicator_column(engine_type),
    tf.feature_column.indicator_column(num_of_cylinders),
    tf.feature_column.indicator_column(fuel_system),
]

In [ ]:
dnnregressor = tf.contrib.learn.DNNRegressor(feature_columns=dnn_features, hidden_units=[50, 30, 10])

In [ ]:
dnnregressor.fit(input_fn=training_input_fn, steps=10000)

In [ ]:
dnnregressor.evaluate(input_fn=eval_input_fn)

In [ ]:
def experiment_fn(run_config, params):
  # This function makes an Experiment, containing an Estimator and inputs for training and evaluation.
  # You can use params and config here to customize the Estimator depending on the cluster or to use
  # hyperparameter tuning.

  # Collect information for training
  return tf.contrib.learn.Experiment(estimator=tf.contrib.learn.LinearRegressor(
                                     feature_columns=linear_features, config=run_config),
                                     train_input_fn=training_input_fn,
                                     train_steps=10000,
                                     eval_input_fn=eval_input_fn)

In [ ]:
# shutil http://www.cnblogs.com/CLTANG/archive/2011/11/15/2249257.html
import shutil
shutil.rmtree("/tmp/output_dir", ignore_errors=True)  # 刪除暫存目錄
tf.contrib.learn.learn_runner.run(experiment_fn, run_config=tf.contrib.learn.RunConfig(model_dir="/tmp/output_dir"))

In [ ]:
peforth.ok()

In [ ]:
scores = dnnregressor.predict(x=dict(eval_data[0:10]))

In [ ]:
[i for i in scores]

In [ ]:
eval_data[0:10]

In [ ]:
_ = dnnregressor.predict(x=dict(eval_data[20:21]))
print(eval_data[20:21])
[i for i in _]

In [ ]:
_ = dnnregressor.predict(x=dict(eval_data[27:28]))
print(eval_data[27:28])
[i for i in _]

In [ ]:
#[k for k,v in dtypes if v == np.float32]
mydict = {'aa':11,'bb':22}
[i for i in mydict]

In [ ]:
[mydict[i] for i in mydict]

In [ ]:
[(n,k) for k,n in mydict.items()]

In [ ]:
mydict.items()

In [ ]:
regressor = tf.contrib.learn.LinearRegressor(feature_columns=linear_features)

In [28]:
make

_HashedCategoricalColumn(key='make', hash_bucket_size=50, dtype=tf.string)

In [18]:
dir(make)

['IdWeightPair',
 '__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_asdict',
 '_fields',
 '_get_sparse_tensors',
 '_make',
 '_num_buckets',
 '_parse_example_spec',
 '_replace',
 '_source',
 '_transform_feature',
 'count',
 'dtype',
 'hash_bucket_size',
 'index',
 'key',
 'name']

In [39]:
make._num_buckets

50

In [25]:
df['make']

0      alfa-romero
1      alfa-romero
2      alfa-romero
3             audi
4             audi
5             audi
6             audi
7             audi
8             audi
10             bmw
11             bmw
12             bmw
13             bmw
14             bmw
15             bmw
16             bmw
17             bmw
18       chevrolet
19       chevrolet
20       chevrolet
21           dodge
22           dodge
23           dodge
24           dodge
25           dodge
26           dodge
27           dodge
28           dodge
29           dodge
30           honda
          ...     
175         toyota
176         toyota
177         toyota
178         toyota
179         toyota
180         toyota
181         toyota
182     volkswagen
183     volkswagen
184     volkswagen
185     volkswagen
186     volkswagen
187     volkswagen
188     volkswagen
189     volkswagen
190     volkswagen
191     volkswagen
192     volkswagen
193     volkswagen
194          volvo
195          volvo
196         